# SENG 550 Final Project 

Luka Petrovic UCID: <br>
Logan Boras UCID: <br> 
Kenny Jeon UCID: 30068677

### 1. Set up environment and Spark ###

In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# uncomment the following if you do not have spark installed in your project
#!wget --no-check-certificate https://dlcdn.apache.org/spark/spark-3.3.3/spark-3.3.3-bin-hadoop3.tgz
#!tar -xvf spark-3.3.3-bin-hadoop3.tgz
!pip install findspark
!pip install numpy
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/training/spark-3.3.3-bin-hadoop3"

You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import findspark
findspark.init()
import pyspark

sc = pyspark.SparkContext(appName="Seng550Project")

### 2. Create directory for input data to live ###

Note that the deerfoot.csv file or whatever input data you are using must be within the data folder then proceed to store it on the hdfs

In [9]:
!mkdir data
!hadoop fs -mkdir -p /inputs
!hadoop fs -put data /inputs

mkdir: cannot create directory ‘data’: File exists
2023-12-18 04:55:15,264 INFO sasl.SaslDataTransferClient: SASL encryption trust check: localHostTrusted = false, remoteHostTrusted = false


### 3. Grab the specific data we want from the data set ###

In [18]:
file = '/inputs/data/deerfoot.csv'

deerfootRDD = (sc.textFile(file, 8)).join(deerfootRDD.zipWithIndex().map(lambda x: str(x[1]) + ': ' + str(x[0])))

TypeError: can only join an iterable

#### 3a. Now we want to get the data only for morning and afternoon rush hour, 8am and 4pm will suffice ####

In [16]:
def getRushHours(RDD):

    rushHours = RDD.split(',')
    return (rushHours[1],rushHours[6],rushHours[13])

print(getRushHours(deerfootRDD.take(1)[0]))


In [17]:
rushHoursRDD = deerfootRDD.map(getRushHours)
print(rushHoursRDD.take(1))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 14.0 failed 1 times, most recent failure: Lost task 1.0 in stage 14.0 (TID 51) (019ac7926b66 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/bitnami/python/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/bitnami/python/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1239, in process
    serializer.dump_stream(out_iter, outfile)
  File "/opt/bitnami/python/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 274, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/opt/bitnami/python/lib/python3.8/site-packages/pyspark/rdd.py", line 2849, in takeUpToNumLeft
    yield next(iterator)


#### 3b. Rush hour generally doesn't happen on weekends, so we need to filter out Saturdays and Sundays ####

In [14]:
# This function was taken from the Spark Basics Tutorial

def removeWeekends(deerfootRDDRecord):
    if deerfootRDDRecord[0]=="Saturday" or deerfootRDDRecord[0]=="Sunday":
        return False
    else:
        return True

weekdayRushHoursRDD = rushHoursRDD.filter(removeWeekends)
#print(weekdayRushHoursRDD.take(5))

### 4. Compute Statistics and Averages

For our analysis, we want to get both the weekly AM and PM averages, as well as the overall average. For a day to be considered "bad", the mean of the AM and PM commute times should be higher than the overall average commute time.

#### 4a. Get day counts

In [15]:
totalCount = weekdayRushHoursRDD.count()
print(totalCount)

# The below might be useful later, uncomment if we need counts of each day in dataset
# perDayRDD = weekdayRushHoursRDD.map(lambda x: (x[0],1))
# perDayCountsRDD = perDayRDD.reduceByKey(lambda x,y:x+y)

# dayCountsList = perDayCountsRDD.collect()
# dayCountsDict = dict(deerfootDayCountsList)

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 10.0 failed 1 times, most recent failure: Lost task 2.0 in stage 10.0 (TID 35) (019ac7926b66 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/bitnami/python/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1247, in main
    process()
  File "/opt/bitnami/python/lib/python3.8/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1237, in process
    out_iter = func(split_index, iterator)
  File "/opt/bitnami/python/lib/python3.8/site-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/opt/bitnami/python/lib/python3.8/site-packages/pyspark/rdd.py", line 5434, in pipeline_func
    return func(split, prev_func(split, iterator))
  Fi

#### 4b. Get overall rush hour average

In [5]:
#testRdd = sc.parallelize([('Monday', '45', '40'), ('Tuesday', '52', '40'), ('Wednesday', '39', '40')])
totalRushHourSum = sc.accumulator(0) # total average


def sum(item):
    global totalRushHourSum
    totalRushHourSum += int(item[1])
    totalRushHourSum += int(item[2])
    
weekdayRushHoursRDD.foreach(sum)
totalRushHourAverage = totalRushHourSum.value/(totalCount*2)
print(totalRushHourAverage)    


42.666666666666664


#### Reduce down am and pm commute times into a daily average

In [11]:
weekdayAverageRushHoursRDD = weekdayRushHoursRDD.map(lambda x: (x[0], (int(x[1])+int(x[2]))/2))
print(weekdayAverageRushHoursRDD.take(2))
                                                    

[('Monday', 42.5), ('Tuesday', 46.0)]


### 5 - Add labels to data

We want to add a "good" or "bad" label to each field. In order to do this, we will compare the daily average commute time with the total average. If it's higher, its a bad day. If its lower, consider it a good day. We will use the integer 1 as good and 0 as bad.

In [13]:
def addLabels(item):
    goodOrBadLabel = 1
    if(item[1] > totalRushHourAverage):
        goodOrBadLabel = 0
        
    return (goodOrBadLabel,item[0], item[1])


finalRDD = weekdayAverageRushHoursRDD.map(addLabels)
print(finalRDD.take(2))

[(1, 'Monday', 42.5), (0, 'Tuesday', 46.0)]


### 6. Set up logistic regression model for prediction

Now that we have our inputs sorted, we can start to make our model

In [17]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression

inputData = finalRDD.toDf("Label", "Weekday", "Avg. Commute")
print(h.schema)

featureColumns = ["Weekday", "Avg. Commute"]
assembler = VectorAssembler(inputCols=featureColumns, outputCol="features")

inputData = assembler.transform(inputData)

trainData, testData = data.randomSplit([0.8, 0.2], seed=21)

ModuleNotFoundError: No module named 'numpy'

In [ ]:
model = LogisticRegression(featuresCol="features", labelCol="label").fit(train_data)


### 7. Evalute Model

Now that we have our model built and trained, we will evaluate it with test data